In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
# Activate the automatic conversion of Pandas DataFrames to R DataFrames
pandas2ri.activate()

### Prepare genotype file, hapmap3+ file:

In [ ]:
# read bed, bim, fam to generate .rds and .bk file
bigsnpr = importr('bigsnpr')
print("Loading genotype data...")
try:
    # Import the R packages
    bigsnpr.snp_readBed('/home/sangnv/Desktop/VGP_10_diseases/target_data/eas.1kgp.hapmap3.annotate.bed')
except:
    pass

R[write to console]: Error: File '/home/sangnv/Desktop/VGP_10_diseases/target_data/eas.1kgp.hapmap3.annotate.bk' already exists.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages



Loading genotype data...


In [ ]:
# Run once for all

r0_code = """
setwd('~/Desktop/VGP_10_diseases/methods/LDpred2/')
library(bigsnpr)

# Load the genotype data
obj.bigSNP <- snp_attach("/home/sangnv/Desktop/VGP_10_diseases/target_data/eas.1kgp.hapmap3.annotate.rds")
G   <- obj.bigSNP$genotypes
CHR <- obj.bigSNP$map$chromosome
POS <- obj.bigSNP$map$physical.pos
NCORES <- nb_cores()
map <- setNames(obj.bigSNP$map[-3], c("chr", "rsid", "pos", "a1", "a0"))
POS2 <- snp_asGeneticPos(CHR, POS, dir = ".", ncores = as.integer(NCORES/2))

# Load hapmap3+ variants
hapmap3 <- readRDS('./map_hm3_plus.rds')

"""

robjects.r(r0_code)

### Run for each disease:

In [26]:
def LDpred2_auto(sumstats_file: str, 
                 output_folder: str,
                 output_name: str):
    
    print("Loading summary statistics...")
    sumstats = pd.read_table(sumstats_file)
    robjects.globalenv['sumstats'] = sumstats

    print("Run R code...")
    r_code = """
    setwd('~/Desktop/VGP_10_diseases/methods/LDpred2/')

    # Merge the summary statistics with hapmap3+
    sumstats <- sumstats[sumstats$rsid %in% hapmap3$rsid, ]

    # SNPs matching
    print("Matching SNPs...")
    df_beta <- snp_match(sumstats, map, join_by_pos = FALSE, match.min.prop=0.01)

    # Calculate LD score
    tmp <- tempfile(tmpdir = "tmp_data")

    print("Calculate LD score in chr:")
    for (chr in 1:22) {
        print(chr)
        ind.chr <- which(df_beta$chr == chr)
        ind.chr2 <- df_beta$`_NUM_ID_`[ind.chr]
        corr0 <- snp_cor(G, 
                        ind.col = ind.chr2, 
                        size = 1 / 1000,
                        infos.pos = POS2[ind.chr2], 
                        ncores = as.integer(NCORES/4))
        if (chr == 1) {
            ld <- Matrix::colSums(corr0^2)
            corr <- as_SFBM(corr0, tmp, compact = TRUE)
        } else {
            ld <- c(ld, Matrix::colSums(corr0^2))
            corr$add_columns(corr0, nrow(corr))
        }
    }
    
    ldsc <- with(df_beta, snp_ldsc(ld, length(ld), chi2 = (beta / beta_se)^2,
                                    sample_size = n_eff, blocks = NULL))
    ldsc_h2_est <- ldsc[["h2"]]
    print(paste0("ldsc_h2 = ", ldsc_h2_est))

    # Perform LDpred2-grid
    set.seed(1)
    print("Run LDpred2-grid...")
    h2_seq <- round(ldsc_h2_est * c(0.7, 1, 1.4), 4)
    p_seq <- signif(seq_log(1e-5, 0.5, length.out = 15), 2)
    params <- expand.grid(p = p_seq, h2 = h2_seq, sparse = c(FALSE, TRUE))
    beta_grid <- snp_ldpred2_grid(corr, df_beta, params, ncores = as.integer(NCORES/4))

    # Perform LDpred2-auto
    set.seed(1)
    print("Run LDpred2-auto...")
    multi_auto <- snp_ldpred2_auto(
        corr, 
        df_beta, 
        h2_init = ldsc_h2_est,
        vec_p_init = seq_log(1e-5, 0.5, length.out = 30), 
        ncores = as.integer(NCORES/4),
        allow_jump_sign = FALSE,
        use_MLE = FALSE,
        shrink_corr = 0.95)

    range <- sapply(multi_auto, function(auto) diff(range(auto$corr_est)))
    keep <- which(range > (0.95 * quantile(range, 0.95, na.rm = TRUE)))
    beta_auto <- rowMeans(sapply(multi_auto[keep], function(auto) auto$beta_est))

    """

    robjects.r(r_code)

    # clear tmp_data memory
    os.system("rm /home/sangnv/Desktop/VGP_10_diseases/methods/LDpred2/tmp_data/*")

    beta_file = os.path.join(output_folder, output_name)
    df_beta = pandas2ri.rpy2py(robjects.globalenv['df_beta'])[['chr', 'rsid', 'a0', 'a1']].reset_index(drop=True)
    df_beta['beta.auto'] = robjects.globalenv['beta_auto']
    beta_grid = pd.DataFrame(robjects.globalenv['beta_grid'], 
                             columns=["beta.grid." + str(i) for i in range(1, 91)])
    df_beta = pd.concat([df_beta, beta_grid], axis=1)
    df_beta.to_csv(beta_file, sep='\t', index=False)
    print(f'Beta file saved in {beta_file}')
    print("Done! \n")

## 1. Breast cancer:

In [ ]:
# BBJ
sumstats_file = "../../sum_stats_data/breast_cancer/hum0197.v3.BBJ.BC.v1/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
            output_folder="../../results/breast_cancer/",
            output_name="LDpred2.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,047,607 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,319 variants have been matched; 532 were flipped and 322,091 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0426356227804056"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/breast_cancer/LDpred2.pst.beta.txt
Done! 



In [7]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/breast_cancer/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/breast_cancer/",
             output_name="LDpred2.shaprs.BBJ.EUR.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,047,607 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,319 variants have been matched; 532 were flipped and 322,091 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0213924853018115"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/breast_cancer/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 2. Colorectal cancer

In [8]:
# BBJ
sumstats_file = "../../sum_stats_data/colorectal_cancer/hum0197.v3.BBJ.CC.v1/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/colorectal_cancer/",
             output_name="LDpred2.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,045,672 variants to be matched.

R[write to console]: 3 ambiguous SNPs have been removed.

R[write to console]: 994,561 variants have been matched; 532 were flipped and 321,207 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0267536927992331"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/colorectal_cancer/LDpred2.pst.beta.txt
Done! 



In [9]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/colorectal_cancer/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/colorectal_cancer/",
             output_name="LDpred2.shaprs.BBJ.EUR.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,045,672 variants to be matched.

R[write to console]: 3 ambiguous SNPs have been removed.

R[write to console]: 994,561 variants have been matched; 532 were flipped and 321,207 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00899433044615795"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/colorectal_cancer/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 3. Gastric cancer

In [10]:
# BBJ
sumstats_file = "../../sum_stats_data/gastric_cancer/hum0197.v3.BBJ.GC.v1/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/gastric_cancer/",
             output_name="LDpred2.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,045,669 variants to be matched.

R[write to console]: 3 ambiguous SNPs have been removed.

R[write to console]: 994,546 variants have been matched; 532 were flipped and 321,197 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0254720472621175"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/gastric_cancer/LDpred2.pst.beta.txt
Done! 



In [11]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/gastric_cancer/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/gastric_cancer/",
             output_name="LDpred2.shaprs.BBJ.EUR.pst.beta.txt")

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,045,669 variants to be matched.

R[write to console]: 3 ambiguous SNPs have been removed.

R[write to console]: 994,546 variants have been matched; 532 were flipped and 321,197 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00608914255296275"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/gastric_cancer/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 4. PD (Parkinson Disease)

In [12]:
# BBJ
sumstats_file = "../../sum_stats_data/pd/gwas.catalog.EAS.GCST90278092/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/pd/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 125,355 variants to be matched.

R[write to console]: 1 ambiguous SNPs have been removed.

R[write to console]: 124,213 variants have been matched; 0 were flipped and 38,467 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.307659186083534"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/pd/LDpred2.pst.beta.txt
Done! 



In [13]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/pd/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/pd/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 125,355 variants to be matched.

R[write to console]: 1 ambiguous SNPs have been removed.

R[write to console]: 124,213 variants have been matched; 0 were flipped and 38,467 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00186726727756638"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/pd/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 5. CKD (Chronic Kidney Disease)

In [14]:
# BBJ
sumstats_file = "../../sum_stats_data/ckd/hum0197.v3.BBJ.CRF.v1/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/ckd/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,048,969 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,641 variants have been matched; 532 were flipped and 322,193 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00504564158627518"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/ckd/LDpred2.pst.beta.txt
Done! 



In [15]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/ckd/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/ckd/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,048,969 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,641 variants have been matched; 532 were flipped and 322,193 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00564195842138073"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/ckd/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 6. Osteoporosis

In [16]:
# BBJ
sumstats_file = "../../sum_stats_data/osteoporosis/hum0197.v3.BBJ.OP.v1/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/osteoporosis/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,048,958 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,607 variants have been matched; 532 were flipped and 322,183 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0197243076031525"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/osteoporosis/LDpred2.pst.beta.txt
Done! 



In [17]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/osteoporosis/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/osteoporosis/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,048,958 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 997,607 variants have been matched; 532 were flipped and 322,183 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.00637127660252187"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/osteoporosis/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 7. CAD (Coronary Artery Disease)

In [18]:
# BBJ
sumstats_file = "../../sum_stats_data/cad/gwas.BBJ.CAD/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/cad/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,019,890 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 985,387 variants have been matched; 489 were flipped and 320,237 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0821971671199656"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/cad/LDpred2.pst.beta.txt
Done! 



In [19]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/cad/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/cad/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,019,890 variants to be matched.

R[write to console]: 4 ambiguous SNPs have been removed.

R[write to console]: 985,387 variants have been matched; 489 were flipped and 320,237 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0408575660316878"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/cad/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 8. Hyperlipidemia

In [20]:
# BBJ
sumstats_file = "../../sum_stats_data/hyperlipidemia/gwas.catalog.EAS.GCST90090994/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/hyperlipidemia/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 265,353 variants to be matched.

R[write to console]: 0 ambiguous SNPs have been removed.

R[write to console]: 264,786 variants have been matched; 0 were flipped and 90,468 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.378578622419308"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/hyperlipidemia/LDpred2.pst.beta.txt
Done! 



In [21]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/hyperlipidemia/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/hyperlipidemia/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 265,353 variants to be matched.

R[write to console]: 0 ambiguous SNPs have been removed.

R[write to console]: 264,786 variants have been matched; 0 were flipped and 90,468 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0519844144876526"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/hyperlipidemia/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 



## 9. Osteoarthritis

In [22]:
# BBJ
sumstats_file = "../../sum_stats_data/osteoarthritis/gwas.catalog.EAS.GCST90134281/gwas.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/osteoarthritis/",
             output_name='LDpred2.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,288,243 variants to be matched.

R[write to console]: 0 ambiguous SNPs have been removed.

R[write to console]: 1,024,280 variants have been matched; 723 were flipped and 16,094 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.715925920377519"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/osteoarthritis/LDpred2.pst.beta.txt
Done! 



In [23]:
# shaPRS (BBJ+EUR) + LDpred2
sumstats_file = "../../sum_stats_data/osteoarthritis/gwas.BBJ.EUR_shaprs.LDpred2.txt"
LDpred2_auto(sumstats_file=sumstats_file,
             output_folder="../../results/osteoarthritis/",
             output_name='LDpred2.shaprs.BBJ.EUR.pst.beta.txt')

Loading summary statistics...
Run R code...
[1] "Matching SNPs..."


R[write to console]: 1,288,243 variants to be matched.

R[write to console]: 0 ambiguous SNPs have been removed.

R[write to console]: 1,024,280 variants have been matched; 723 were flipped and 16,094 were reversed.



[1] "Calculate LD score in chr:"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] "ldsc_h2 = 0.0520141584152062"
[1] "Run LDpred2-grid..."
[1] "Run LDpred2-auto..."
Beta file saved in ../../results/osteoarthritis/LDpred2.shaprs.BBJ.EUR.pst.beta.txt
Done! 

